In [1]:
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
from model_arch.run_train import *
from utils.step_sample import create_named_schedule_sampler
from model_arch.train import TrainLoop
from utils.data import load_data_text
from model_arch.tokenizer import load_tokenizer, load_model_emb
from model_arch.sampling import sampling

from transformers import AutoTokenizer, PreTrainedTokenizerFast, BertTokenizerFast, set_seed
import json, torch
from utils import dist_util
from functools import partial
import pickle
import random
from datetime import datetime

In [53]:
dist_util.clear_cache()

In [42]:
lr=0.00005
batch_size=30
val_batch_size=20
microbatch=10
epochs=20_000
eval_interval=10
ema_rate='0.9999' 
schedule_sampler='uniform'
diffusion_steps=1000
noise_schedule='sqrt'
vocab='custom'
use_plm_init='no' # embedding in transformer
vocab_size=0
config_name='bert-base-uncased'
seq_len=128
hidden_t_dim=128
hidden_dim=128
dropout=0.1
seed=10275679
weight_decay=0.0
predict_xstart=True
rescale_timesteps=True
emb_scale_factor=1.0

In [43]:
regular_data_dir='data'
data_player_dir='data/with_player'
comedies_data_dir='data/comedies_only'

# set the data directory
data_dir=comedies_data_dir

In [44]:
set_seed(seed)

In [45]:
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [46]:
model_weight, tokenizer = load_model_emb(hidden_dim, tokenizer)

In [47]:
model_weight

Embedding(30268, 128)

In [48]:
## very very important to set this!!!!!
vocab_size = tokenizer.vocab_size
vocab_size

30268

In [49]:
data = load_data_text(
        batch_size=batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        model_emb=model_weight # use model's weights as init
    )

val = load_data_text(
        batch_size=val_batch_size,
        seq_len=seq_len,
        data_dir=data_dir,
        loaded_vocab=tokenizer,
        split='valid',
        model_emb=model_weight, # use model's weights as init
    )

############################## 
Loading text data...
############################## 
Loading dataset from data/comedies_only...
### Loading form the TRAIN set...
### Data samples...
 ['i would not have my right rosalind of this mind, for, i protest, her frown might kill me.', 'i thought, by your readiness in the office, you had continued in it some time. you say, seven years together?'] ['by this hand, it will not kill a fly. but come, now i will be your rosalind in a more coming-on disposition, and ask me what you will. i will grant it.', 'and a half, sir.']
RAM used: 5454.00 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 8706
})
RAM used: 5458.66 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/8706 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 8706
})
### tokenized_datasets...example [2, 31, 241, 125, 150, 105, 755, 2841, 94, 138, 727, 10, 115, 10, 31, 2566, 10, 175, 2191, 526, 803, 117, 12, 3]
RAM used: 5459.00 MB


merge and mask:   0%|          | 0/8706 [00:00<?, ? examples/s]

RAM used: 5471.64 MB


...

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 2167
})
### tokenized_datasets...example [2, 88, 105, 10193, 10, 222, 164, 89, 159, 941, 117, 159, 31, 144, 10, 146, 254, 31, 251, 105, 1485, 88, 105, 13299, 12, 3]
RAM used: 5471.64 MB


merge and mask:   0%|          | 0/2167 [00:00<?, ? examples/s]

RAM used: 5475.55 MB


padding:   0%|          | 0/2167 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 2167
}) padded dataset
RAM used: 5484.27 MB
RAM used: 5484.41 MB


In [50]:
diffusion = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, diffusion_steps),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

best_model_fp = 'models/results/final_model_df1000.pkl'
with open(best_model_fp, 'rb') as handle:
    model = pickle.load(handle)

In [51]:
pytorch_total_params = sum(p.numel() for p in model.parameters())
pytorch_total_params

91192508

In [52]:
schedule_sampler = create_named_schedule_sampler('uniform', diffusion)

model.to(dist_util.dev())

train_loss, val_loss = TrainLoop(
                            model=model,
                            diffusion=diffusion,
                            data=data,
                            batch_size=batch_size,
                            microbatch=microbatch,
                            lr=lr,
                            ema_rate=ema_rate,
                            schedule_sampler=schedule_sampler,
                            weight_decay=weight_decay,
                            epochs=epochs,
                            eval_data=val,
                            eval_interval=eval_interval,
                            warm_up_steps=50,
                            use_llrd=True,
                            llrd_rate=0.9
                        ).run_loop()

dt = datetime.now().strftime("%m%d")
pickle.dump(model, open(f"models/{dt}/final_model_df{diffusion_steps}.pkl", 'wb'))



======== Using Layer-wise Learning Rate Decay with AdamW ========


name: word_embedding.weight, lr: 5e-05
name: lm_head.bias, lr: 5e-05
name: time_embed.0.weight, lr: 5e-05
name: time_embed.0.bias, lr: 5e-05
name: time_embed.2.weight, lr: 5e-05
name: time_embed.2.bias, lr: 5e-05
name: input_up_proj.0.weight, lr: 5e-05
name: input_up_proj.0.bias, lr: 5e-05
name: input_up_proj.2.weight, lr: 5e-05
name: input_up_proj.2.bias, lr: 5e-05
name: input_transformers.layer.0.attention.self.query.weight, lr: 5.555555555555556e-05
name: input_transformers.layer.0.attention.self.query.bias, lr: 5.555555555555556e-05
name: input_transformers.layer.0.attention.self.key.weight, lr: 5.555555555555556e-05
name: input_transformers.layer.0.attention.self.key.bias, lr: 5.555555555555556e-05
name: input_transformers.layer.0.attention.self.value.weight, lr: 5.555555555555556e-05
name: input_transformers.layer.0.attention.self.value.bias, lr: 5.555555555555556e-05
name: input_transformers.layer.0.attention.

Epoch 0/20000 Training Loss: 0.046861130744218826
Epoch 0/20000 Validation Loss: 0.04647196829319
Cleared directory to save new best model.
============>Saving current best model with min_val_loss=0.04647196829319<=============
Epoch 1/20000 Training Loss: 0.06575585156679153
Epoch 2/20000 Training Loss: 0.06334967166185379
Epoch 3/20000 Training Loss: 0.05192740634083748
Epoch 4/20000 Training Loss: 0.050199612975120544
Epoch 5/20000 Training Loss: 0.07412896305322647
Epoch 6/20000 Training Loss: 0.04815239831805229
Epoch 7/20000 Training Loss: 0.05086776614189148
Epoch 8/20000 Training Loss: 0.059878621250391006
Epoch 9/20000 Training Loss: 0.0537346750497818
Epoch 10/20000 Training Loss: 0.06251121312379837
Epoch 10/20000 Validation Loss: 0.060411807149648666
Epoch 11/20000 Training Loss: 0.04301084578037262
Epoch 12/20000 Training Loss: 0.05401583015918732
Epoch 13/20000 Training Loss: 0.06528591364622116
Epoch 14/20000 Training Loss: 0.04747943580150604
Epoch 15/20000 Training Los

Epoch 138/20000 Training Loss: 0.0719294548034668
Epoch 139/20000 Training Loss: 0.0912608802318573
Epoch 140/20000 Training Loss: 0.06126176193356514
Epoch 140/20000 Validation Loss: 0.05715722590684891
Epoch 141/20000 Training Loss: 0.054287731647491455
Epoch 142/20000 Training Loss: 0.05684271082282066
Epoch 143/20000 Training Loss: 0.06864909082651138
Epoch 144/20000 Training Loss: 0.06541838496923447
Epoch 145/20000 Training Loss: 0.07817097753286362
Epoch 146/20000 Training Loss: 0.07764425873756409
Epoch 147/20000 Training Loss: 0.0762665644288063
Epoch 148/20000 Training Loss: 0.06457202881574631
Epoch 149/20000 Training Loss: 0.05994972586631775
Epoch 150/20000 Training Loss: 0.0675877258181572
Epoch 150/20000 Validation Loss: 0.042040929198265076
Epoch 151/20000 Training Loss: 0.055356044322252274
Epoch 152/20000 Training Loss: 0.04870573803782463
Epoch 153/20000 Training Loss: 0.051883865147829056
Epoch 154/20000 Training Loss: 0.053221434354782104
Epoch 155/20000 Training L

Epoch 283/20000 Training Loss: 0.05138108506798744
Epoch 284/20000 Training Loss: 0.05591554939746857
Epoch 285/20000 Training Loss: 0.04297414422035217
Epoch 286/20000 Training Loss: 0.05082109197974205
Epoch 287/20000 Training Loss: 0.0678977370262146
Epoch 288/20000 Training Loss: 0.05472026392817497
Epoch 289/20000 Training Loss: 0.06693508476018906
Epoch 290/20000 Training Loss: 0.05370793491601944
Epoch 290/20000 Validation Loss: 0.05952521413564682
Epoch 291/20000 Training Loss: 0.04837982356548309
Epoch 292/20000 Training Loss: 0.06354865431785583
Epoch 293/20000 Training Loss: 0.06321483105421066
Epoch 294/20000 Training Loss: 0.06408102065324783
Epoch 295/20000 Training Loss: 0.053368452936410904
Epoch 296/20000 Training Loss: 0.05590392276644707
Epoch 297/20000 Training Loss: 0.052984271198511124
Epoch 298/20000 Training Loss: 0.05145363137125969
Epoch 299/20000 Training Loss: 0.059255439788103104
Epoch 300/20000 Training Loss: 0.059561748057603836
Epoch 300/20000 Validation

Epoch 429/20000 Training Loss: 0.05687728524208069
Epoch 430/20000 Training Loss: 0.06097899004817009
Epoch 430/20000 Validation Loss: 0.07599315792322159
Epoch 431/20000 Training Loss: 0.06017455458641052
Epoch 432/20000 Training Loss: 0.04742817580699921
Epoch 433/20000 Training Loss: 0.04477252438664436
Epoch 434/20000 Training Loss: 0.06840742379426956
Epoch 435/20000 Training Loss: 0.06796757131814957
Epoch 436/20000 Training Loss: 0.05977143719792366
Epoch 437/20000 Training Loss: 0.06639730930328369
Epoch 438/20000 Training Loss: 0.08336412161588669
Epoch 439/20000 Training Loss: 0.08374893665313721
Epoch 440/20000 Training Loss: 0.07099553197622299
Epoch 440/20000 Validation Loss: 0.06411202251911163
Epoch 441/20000 Training Loss: 0.06137414649128914
Epoch 442/20000 Training Loss: 0.06273692101240158
Epoch 443/20000 Training Loss: 0.046366069465875626
Epoch 444/20000 Training Loss: 0.06809302419424057
Epoch 445/20000 Training Loss: 0.05911196395754814
Epoch 446/20000 Training L

...

Epoch 19373/20000 Training Loss: 0.05858529731631279
Epoch 19374/20000 Training Loss: 0.04808008670806885
Epoch 19375/20000 Training Loss: 0.0631568506360054
Epoch 19376/20000 Training Loss: 0.046521883457899094
Epoch 19377/20000 Training Loss: 0.04549521580338478
Epoch 19378/20000 Training Loss: 0.05166604742407799
Epoch 19379/20000 Training Loss: 0.049005065113306046
Epoch 19380/20000 Training Loss: 0.044495489448308945
Epoch 19380/20000 Validation Loss: 0.0511292889714241
Epoch 19381/20000 Training Loss: 0.0557762049138546
Epoch 19382/20000 Training Loss: 0.03693823516368866
Epoch 19383/20000 Training Loss: 0.058072347193956375
Epoch 19384/20000 Training Loss: 0.049296215176582336
Epoch 19385/20000 Training Loss: 0.07018435001373291
Epoch 19386/20000 Training Loss: 0.049209654331207275
Epoch 19387/20000 Training Loss: 0.04019713029265404
Epoch 19388/20000 Training Loss: 0.05760771036148071
Epoch 19389/20000 Training Loss: 0.057311952114105225
Epoch 19390/20000 Training Loss: 0.04834

Epoch 19513/20000 Training Loss: 0.05640801787376404
Epoch 19514/20000 Training Loss: 0.06311284750699997
Epoch 19515/20000 Training Loss: 0.05255197361111641
Epoch 19516/20000 Training Loss: 0.0512322336435318
Epoch 19517/20000 Training Loss: 0.0620715469121933
Epoch 19518/20000 Training Loss: 0.04116389527916908
Epoch 19519/20000 Training Loss: 0.05378140136599541
Epoch 19520/20000 Training Loss: 0.046021003276109695
Epoch 19520/20000 Validation Loss: 0.04212082549929619
Epoch 19521/20000 Training Loss: 0.04954395070672035
Epoch 19522/20000 Training Loss: 0.053962502628564835
Epoch 19523/20000 Training Loss: 0.05720418691635132
Epoch 19524/20000 Training Loss: 0.046745166182518005
Epoch 19525/20000 Training Loss: 0.05976935848593712
Epoch 19526/20000 Training Loss: 0.045290905982255936
Epoch 19527/20000 Training Loss: 0.054555606096982956
Epoch 19528/20000 Training Loss: 0.05915170535445213
Epoch 19529/20000 Training Loss: 0.05481760576367378
Epoch 19530/20000 Training Loss: 0.054817

Epoch 19653/20000 Training Loss: 0.056324247270822525
Epoch 19654/20000 Training Loss: 0.03472547605633736
Epoch 19655/20000 Training Loss: 0.07060442119836807
Epoch 19656/20000 Training Loss: 0.05643632635474205
Epoch 19657/20000 Training Loss: 0.06211816146969795
Epoch 19658/20000 Training Loss: 0.055952038615942
Epoch 19659/20000 Training Loss: 0.045102447271347046
Epoch 19660/20000 Training Loss: 0.04069503769278526
Epoch 19660/20000 Validation Loss: 0.051585908979177475
Epoch 19661/20000 Training Loss: 0.05085625872015953
Epoch 19662/20000 Training Loss: 0.03243439644575119
Epoch 19663/20000 Training Loss: 0.05878196284174919
Epoch 19664/20000 Training Loss: 0.047049056738615036
Epoch 19665/20000 Training Loss: 0.05123193562030792
Epoch 19666/20000 Training Loss: 0.06370700895786285
Epoch 19667/20000 Training Loss: 0.047142621129751205
Epoch 19668/20000 Training Loss: 0.04465031623840332
Epoch 19669/20000 Training Loss: 0.04451678693294525
Epoch 19670/20000 Training Loss: 0.054906

Epoch 19793/20000 Training Loss: 0.05084219574928284
Epoch 19794/20000 Training Loss: 0.054430484771728516
Epoch 19795/20000 Training Loss: 0.042294543236494064
Epoch 19796/20000 Training Loss: 0.0658591166138649
Epoch 19797/20000 Training Loss: 0.04864148423075676
Epoch 19798/20000 Training Loss: 0.05957349017262459
Epoch 19799/20000 Training Loss: 0.041492052376270294
Epoch 19800/20000 Training Loss: 0.062048688530921936
Epoch 19800/20000 Validation Loss: 0.07272925972938538
Epoch 19801/20000 Training Loss: 0.06296298652887344
Epoch 19802/20000 Training Loss: 0.05441347137093544
Epoch 19803/20000 Training Loss: 0.05729318782687187
Epoch 19804/20000 Training Loss: 0.046053480356931686
Epoch 19805/20000 Training Loss: 0.0426378957927227
Epoch 19806/20000 Training Loss: 0.05313101410865784
Epoch 19807/20000 Training Loss: 0.054423198103904724
Epoch 19808/20000 Training Loss: 0.05623513460159302
Epoch 19809/20000 Training Loss: 0.042432863265275955
Epoch 19810/20000 Training Loss: 0.0494

Epoch 19933/20000 Training Loss: 0.04936782643198967
Epoch 19934/20000 Training Loss: 0.057489026337862015
Epoch 19935/20000 Training Loss: 0.0613127239048481
Epoch 19936/20000 Training Loss: 0.05819815397262573
Epoch 19937/20000 Training Loss: 0.05381517484784126
Epoch 19938/20000 Training Loss: 0.05978461727499962
Epoch 19939/20000 Training Loss: 0.052602630108594894
Epoch 19940/20000 Training Loss: 0.04567664861679077
Epoch 19940/20000 Validation Loss: 0.06594572961330414
Epoch 19941/20000 Training Loss: 0.05016736686229706
Epoch 19942/20000 Training Loss: 0.06132359802722931
Epoch 19943/20000 Training Loss: 0.05086119845509529
Epoch 19944/20000 Training Loss: 0.0458657331764698
Epoch 19945/20000 Training Loss: 0.04054535925388336
Epoch 19946/20000 Training Loss: 0.07431098818778992
Epoch 19947/20000 Training Loss: 0.04856088384985924
Epoch 19948/20000 Training Loss: 0.048321183770895004
Epoch 19949/20000 Training Loss: 0.05694011226296425
Epoch 19950/20000 Training Loss: 0.05255967

In [ ]:
# plot(train_loss, val_loss)

In [ ]:
# param_names = []
# for i, (name, param) in enumerate(model.named_parameters()):
#     param_names.append(name)
#     print(f'{i}: {name} {param.requires_grad}')

# Generating sequences

In [54]:
best_model_fp = f'models/0311/final_model_df1000.pkl'
with open(best_model_fp, 'rb') as handle:
    best_model = pickle.load(handle)

In [56]:
diffusion = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 1000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

In [57]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=64, 
                                                           show_intermediate_results=False)

num_proc must be <= 2. Reducing num_proc to 2 for dataset of size 2.


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['call her forth to me', 'no, sir, i live by the church.'] ['', 'art thou a churchman?']
RAM used: 5540.30 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 2
})
RAM used: 5540.30 MB


Running tokenizer on dataset (num_proc=2):   0%|          | 0/2 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 2
})
### tokenized_datasets...example [2, 439, 175, 769, 88, 117, 3]
RAM used: 5541.81 MB


merge and mask:   0%|          | 0/2 [00:00<?, ? examples/s]

RAM used: 5541.81 MB


padding:   0%|          | 0/2 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 2
}) padded dataset
RAM used: 5541.81 MB
RAM used: 5541.81 MB
### End of reading iteration...


  0%|          | 0/1000 [00:00<?, ?it/s]

Generating 20 sentences takes 5 minutes

In [58]:
word_lst_source

['[CLS] call her forth to me [SEP] [SEP]',
 '[CLS] no, sir, i live by the church. [SEP] [SEP]']

In [59]:
word_lst_recover

['[CLS] i beseech love to soul. [SEP]',
 '[CLS] what? is one, i pray but here of the duke. [SEP]']